In [ ]:
import datetime
from pathlib import Path
import sys

from ppadb.client import Client as AdbClient
import numpy as np

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.utils.frame_widget import FrameWidget
from skrgbd.data.io import imgio
from skrgbd.utils.logging import logger, tqdm
from skrgbd.devices.phone import convert_bin_to_img, LeftPhone, RightPhone, scan_root

RightPhone.serial = RightPhone.usb_serial
LeftPhone.serial = LeftPhone.usb_serial

In [ ]:
today = datetime.datetime.now().strftime('%y_%m_%d')
logfile = f'/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/calibration/logs/{today}.log'
logger.prepare_for_scanning(logfile)

In [ ]:
client = AdbClient(host='127.0.0.1', port=5037)
# cam = RightPhone(client)
cam = LeftPhone(client)
cam.start()

In [ ]:
def prepare(self):
    self.clean_scans()
    self.prepare_scan_dir('depth_wall')

    
def save_imgs(self, i):
    path = f'{scan_root}/depth_wall/{self.name}/{i:06}'
    self.save_ir(path)
    self.save_depth(path)


def read_img(self, i, mode='ir'):
    path = f'{scan_root}/depth_wall/{self.name}/{i:06}_{mode}.bin'
    img = self.read_file(path)
    img = np.frombuffer(img, dtype=np.uint16)
    img = img.reshape(180, 240)
    img = np.rot90(img, 2)
    return img

In [ ]:
prepare(cam)

img_i = 0
save_imgs(cam, img_i)
# widget = FrameWidget(lambda: read_img(cam, img_i, 'depth'), (8, 5), cmap='jet', vmin=500, vmax=1200)
widget = FrameWidget(lambda: read_img(cam, img_i, 'ir'), (8, 5), cmap='gray')
widget.update()
widget.image

In [ ]:
img_i = 0
while True:
    save_imgs(cam, img_i)
    widget.update()
    img_i += 1

In [ ]:
cam.pull_scans('/mnt/data/sk3d/calibration/images', rm=False)
convert_bin_to_img('/mnt/data/sk3d/calibration/images')

In [ ]:
cam.clean_scans()

In [ ]:
cam.stop()